In [2]:
import pandas as pd


df = pd.read_csv('/content/ChronicKidneyDisease_EHRs_from_AbuDhabi.csv')



In [3]:
print("First 5 rows of the dataset:")
print(df.head())
print("\nDataset shape:", df.shape)
print("\nData types and non-null counts:")
print(df.info())




First 5 rows of the dataset:
   Sex  AgeBaseline  HistoryDiabetes  HistoryCHD  HistoryVascular  \
0    0           64                0           0                0   
1    0           52                0           0                0   
2    0           56                0           0                0   
3    0           58                0           0                0   
4    0           63                1           0                0   

   HistorySmoking  HistoryHTN   HistoryDLD  HistoryObesity  DLDmeds  ...  \
0               0            1           1               1        1  ...   
1               0            1           1               1        0  ...   
2               0            1           1               1        1  ...   
3               0            0           1               1        1  ...   
4               0            1           1               1        1  ...   

   ACEIARB  CholesterolBaseline  CreatinineBaseline  eGFRBaseline  \
0        0                  4.

In [4]:
print("\nMissing values in each column:")
print(df.isnull().sum())




Missing values in each column:
Sex                    0
AgeBaseline            0
HistoryDiabetes        0
HistoryCHD             0
HistoryVascular        0
HistorySmoking         0
HistoryHTN             0
HistoryDLD             0
HistoryObesity         0
DLDmeds                0
DMmeds                 0
HTNmeds                0
ACEIARB                0
CholesterolBaseline    0
CreatinineBaseline     0
eGFRBaseline           0
sBPBaseline            0
dBPBaseline            0
BMIBaseline            0
TimeToEventMonths      0
EventCKD35             0
TIME_YEAR              0
dtype: int64


In [5]:

print("\nSummary statistics:")
print(df.describe())





Summary statistics:
              Sex  AgeBaseline  HistoryDiabetes  HistoryCHD  HistoryVascular  \
count  491.000000   491.000000       491.000000  491.000000       491.000000   
mean     0.509165    53.203666         0.437882    0.091650         0.059063   
std      0.500426    13.821282         0.496632    0.288825         0.235983   
min      0.000000    23.000000         0.000000    0.000000         0.000000   
25%      0.000000    44.000000         0.000000    0.000000         0.000000   
50%      1.000000    54.000000         0.000000    0.000000         0.000000   
75%      1.000000    64.000000         1.000000    0.000000         0.000000   
max      1.000000    89.000000         1.000000    1.000000         1.000000   

       HistorySmoking  HistoryHTN   HistoryDLD  HistoryObesity     DLDmeds  \
count      491.000000   491.000000  491.000000      491.000000  491.000000   
mean         0.152749     0.682281    0.645621        0.505092    0.551935   
std          0.360113   

In [6]:
print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")

print("\nUnique values in 'Sex' column:")
print(df['Sex'].value_counts())




Number of duplicate rows: 0

Unique values in 'Sex' column:
Sex
1    250
0    241
Name: count, dtype: int64


In [7]:

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')


numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())


df = df.drop_duplicates()


binary_cols = [col for col in df.columns if col.startswith('history') or col in ['eventckd35', 'dldmeds', 'dmmeds', 'htnmeds', 'aceiarb']]
for col in binary_cols:
    df[col] = df[col].astype(int)


df['egfrbaseline'] = df['egfrbaseline'].clip(lower=0, upper=200)

print("\nData types after cleaning:")
print(df.info())




Data types after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   sex                  491 non-null    int64  
 1   agebaseline          491 non-null    int64  
 2   historydiabetes      491 non-null    int64  
 3   historychd           491 non-null    int64  
 4   historyvascular      491 non-null    int64  
 5   historysmoking       491 non-null    int64  
 6   historyhtn           491 non-null    int64  
 7   historydld           491 non-null    int64  
 8   historyobesity       491 non-null    int64  
 9   dldmeds              491 non-null    int64  
 10  dmmeds               491 non-null    int64  
 11  htnmeds              491 non-null    int64  
 12  aceiarb              491 non-null    int64  
 13  cholesterolbaseline  491 non-null    float64
 14  creatininebaseline   491 non-null    float64
 15  egfrbaseline

In [8]:
bins = [0, 40, 60, float('inf')]
labels = ['young', 'middle', 'senior']
df['age_group'] = pd.cut(df['agebaseline'], bins=bins, labels=labels)


def min_max_scale(col):
    return (df[col] - df[col].min()) / (df[col].max() - df[col].min())
df['sbpbaseline_scaled'] = min_max_scale('sbpbaseline')
df['dbpbaseline_scaled'] = min_max_scale('dbpbaseline')

grouped = df.groupby(['sex', 'historydiabetes']).agg({'egfrbaseline': 'mean'}).reset_index()
print("\nAverage eGFR by Sex and Diabetes History:")
print(grouped)


df_htn = df[df['historyhtn'] == 1]

print("\nFirst 5 rows after transformations:")
print(df.head())




Average eGFR by Sex and Diabetes History:
   sex  historydiabetes  egfrbaseline
0    0                0    104.471831
1    0                1     95.455556
2    1                0    100.027612
3    1                1     90.031034

First 5 rows after transformations:
   sex  agebaseline  historydiabetes  historychd  historyvascular  \
0    0           64                0           0                0   
1    0           52                0           0                0   
2    0           56                0           0                0   
3    0           58                0           0                0   
4    0           63                1           0                0   

   historysmoking  historyhtn  historydld  historyobesity  dldmeds  ...  \
0               0           1           1               1        1  ...   
1               0           1           1               1        0  ...   
2               0           1           1               1        1  ...   
3              

In [10]:

df.to_csv('ckd_wrangled.csv', index=False)


grouped.to_csv('ckd_summary_by_sex_diabetes.csv', index=False)

print("\nWrangled dataset saved as 'ckd_wrangled.csv'")
print("Summary saved as 'ckd_summary_by_sex_diabetes.csv'")


Wrangled dataset saved as 'ckd_wrangled.csv'
Summary saved as 'ckd_summary_by_sex_diabetes.csv'
